In [1]:
import matplotlib.pyplot as plt
import torch
# Make tensorflow not take over the entire GPU memory

if torch.cuda.is_available():
    # Set memory growth behavior (manually or automatically managed by CUDA)
    for i in range(torch.cuda.device_count()):
        # Set the memory fraction (optional, defaults to 1.0 meaning use all available memory)
        torch.cuda.set_per_process_memory_fraction(1.0, i)
        
        # Optionally, you can also clear unused memory (this is the closest thing to memory growth in PyTorch)
        torch.cuda.empty_cache()

from torchga.torchga import GeometricAlgebra
from torchga.blades import BladeKind
from torchga.layers import GeometricProductConv1D

/home/a/albertopepe/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [ ]:
ga = GeometricAlgebra([0, 1, 1, 1])

batch_size = 2
sequence_length = 8
c_in = 3
c_out = 4
kernel_size = 3

a = ga.from_tensor_with_kind(torch.ones([batch_size, sequence_length, c_in, ga.num_blades]), BladeKind.MV)
k = ga.from_tensor_with_kind(torch.ones([kernel_size, c_in, c_out, ga.num_blades]), BladeKind.MV)

y = ga.geom_conv1d(a, k, 2, "SAME")

print(y.shape)
print(y)

torch.Size([2, 4, 4, 16])
tensor([[[[  0., -36.,   0.,  36.,   0.,  36.,  36., -36.,  36.,   0.,  36.,
            36.,  36.,  36.,  36.,  36.],
          [  0., -36.,   0.,  36.,   0.,  36.,  36., -36.,  36.,   0.,  36.,
            36.,  36.,  36.,  36.,  36.],
          [  0., -36.,   0.,  36.,   0.,  36.,  36., -36.,  36.,   0.,  36.,
            36.,  36.,  36.,  36.,  36.],
          [  0., -36.,   0.,  36.,   0.,  36.,  36., -36.,  36.,   0.,  36.,
            36.,  36.,  36.,  36.,  36.]],

         [[  0., -36.,   0.,  36.,   0.,  36.,  36., -36.,  36.,   0.,  36.,
            36.,  36.,  36.,  36.,  36.],
          [  0., -36.,   0.,  36.,   0.,  36.,  36., -36.,  36.,   0.,  36.,
            36.,  36.,  36.,  36.,  36.],
          [  0., -36.,   0.,  36.,   0.,  36.,  36., -36.,  36.,   0.,  36.,
            36.,  36.,  36.,  36.,  36.],
          [  0., -36.,   0.,  36.,   0.,  36.,  36., -36.,  36.,   0.,  36.,
            36.,  36.,  36.,  36.,  36.]],

         [[  0., -

In [10]:
ga.blades

['',
 '0',
 '1',
 '2',
 '3',
 '01',
 '02',
 '03',
 '12',
 '13',
 '23',
 '012',
 '013',
 '023',
 '123',
 '0123']

In [3]:
print(a.shape)

torch.Size([2, 8, 3, 16])


In [5]:
mv_indices = torch.arange(0, ga.num_blades, dtype=torch.int64)

conv_layer = GeometricProductConv1D(
    ga, num_input_filters=c_in, num_output_filters=c_out, kernel_size=kernel_size, stride=1, padding="SAME",
    blade_indices_kernel=torch.arange(0, ga.num_blades, dtype=torch.int64),
    blade_indices_bias=torch.arange(0, ga.num_blades, dtype=torch.int64)
)

y2 = conv_layer(a)
print(y2.shape)
ga.print(y2)
ga.print(y2[0, 0, 0])

torch.Size([2, 8, 4, 16])
MultiVector[batch_shape=torch.Size([2, 8, 4])]
MultiVector[-0.49*1 + 0.94*e_0 + -0.55*e_1 + -0.18*e_2 + -0.97*e_3 + -0.44*e_01 + -0.98*e_02 + -0.58*e_03 + -0.08*e_12 + -0.05*e_13 + -0.00*e_23 + 0.49*e_012 + -0.23*e_013 + 0.62*e_023 + -0.09*e_123 + -0.71*e_0123]


In [7]:
ga = GeometricAlgebra([1, 1, 1, 1])

batch_size = 2
sequence_length = 6
c_in = 3
c_out = 4
kernel_size = 3


a = ga.from_tensor_with_kind(torch.ones([batch_size, sequence_length, c_in, ga.num_blades]), BladeKind.BIVECTOR)
print(a.shape)

RuntimeError: index_add_(): Number of indices (6) should be equal to source.size(dim): (16), for dim: 0